In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import xlwings as xw

#抓取奇摩股市到指定table的資訊
res = requests.get("https://tw.stock.yahoo.com/d/i/rank.php?t=vol&e=tse")
html = BeautifulSoup(res.text, "html.parser")
table = html.find("table", {"cellspacing": "1"})

#建立空的list
num_lst = []
name_lst = []
price_lst = []
change_lst = []
max_lst = []
min_lst = []
spread_lst = []
deal_num_lst = []
deal_value_lst = []

#建立dictionary，方便後續將字典轉換成dataframe匯入excel
stock_dict = {"stock_num": num_lst, "stock_name": name_lst, "stock_price": price_lst, "stock_change": change_lst,
              "stock_max": max_lst, "stock_min": min_lst, "stock_spread": spread_lst, "stock_deal_num": deal_num_lst,
              "stock_deal_value": deal_value_lst}

#使用迴圈將成交量前二十名的資訊爬取出來，並存入到對應的list中
for i in range(1, 21):
    stock = table.findAll("tr")[i]
    stock_num = stock.findAll("td")[0].text
    stock_name = stock.findAll("td")[1].text
    stock_price = stock.findAll("td")[2].text
    stock_change = stock.findAll("td")[3].text
    stock_max = stock.findAll("td")[5].text
    stock_min = stock.findAll("td")[6].text
    stock_spread = stock.findAll("td")[7].text
    stock_deal_num = stock.findAll("td")[8].text
    stock_deal_value = stock.findAll("td")[9].text
    num_lst.append(stock_num)
    name_lst.append(stock_name)
    price_lst.append(stock_price)
    change_lst.append(stock_change)
    max_lst.append(stock_max)
    min_lst.append(stock_min)
    spread_lst.append(stock_spread)
    deal_num_lst.append(stock_deal_num)
    deal_value_lst.append(stock_deal_value)
    
#將dictionary轉換成dataframe的資料結構，並將排名設定成索引值
df = pd.DataFrame(stock_dict)
df.set_index("stock_num", inplace=True)

#將excel的標題設定成list
headers = ["名次", "股票代號/名稱", "成交價", "漲跌", "最高", "最低", "價差", "成交張數", "成交值(億)"]

#建立一個新的檔案
new_wb = xw.Book()
sheet = new_wb.sheets.add(name="單日熱門股進出")
sheet_fg = new_wb.sheets.add(name="上市外資買超排行")
new_wb.sheets["工作表1"].delete()
sheet.range("A1").value = df
sheet.range("A1").value = headers

#爬取外資買超排行資料，並寫入工作表上市外資買超排行
res_fg = requests.get("https://tw.stock.yahoo.com/d/i/fgbuy_tse.html")
html = BeautifulSoup(res_fg.text, "html.parser")
table = html.find("table", {"cellspacing": "1"})
fg_headers = ["名次", "股票代號/名稱", "成交價", "漲跌", "買超張數", "外資持股張數", "外資持股比率"]

sheet_fg.range("A1").value = fg_headers

for i in range(1, 31):
    fg_stock = table.findAll("tr")[i]
    fg_num = fg_stock.findAll("td")[0].text
    fg_name = fg_stock.findAll("td")[1].text
    fg_price = fg_stock.findAll("td")[2].text
    fg_change = fg_stock.findAll("td")[3].text
    fg_buy_more = fg_stock.findAll("td")[4].text
    fg_own = fg_stock.findAll("td")[5].text
    fg_own_rate = fg_stock.findAll("td")[6].text
    
    sheet_fg.range(f"A{i+1}").value = fg_num
    sheet_fg.range(f"B{i+1}").value = fg_name
    sheet_fg.range(f"C{i+1}").value = fg_price
    sheet_fg.range(f"D{i+1}").value = fg_change
    sheet_fg.range(f"E{i+1}").value = fg_buy_more
    sheet_fg.range(f"F{i+1}").value = fg_own
    sheet_fg.range(f"G{i+1}").value = fg_own_rate
    
new_wb.save("台股交易量與外資買超排行.xlxs")